# Работа с данными бизнеса в ClickHouse

## Цель проекта: 
Проведение анализа метрик и паттернов использования сервиса Яндекс Книги для выявления ключевых тенденций и оптимизации пользовательского опыта.

## Задание 1

Для начала продакт-менеджер хочет понять, где сервис пользуется наибольшей популярностью. Выведите топ-20 городов и регионов России по суммарному количеству прочитанных и прослушанных часов любого контента с мобильных устройств. Для каждой из платформ — iOS и Android — добавьте отдельный столбец с длительностью. Результат должен выглядеть так: город, общая длительность прочитанного и прослушанного контента, длительность на iOS, длительность на Android. Значения округлите до целых чисел для лучшей читаемости. Из выдачи также исключите федеральные округа — оставьте только города и области.

In [ ]:
SELECT usage_geo_id_name,
       round(sum(hours)) AS sum_hours,
       round(sumIf(hours, usage_platform_ru = 'Букмейт iOS')) AS sum_hours_ios,
       round(sumIf(hours, usage_platform_ru = 'Букмейт Android')) AS sum_hours_android
FROM source_db.audition
WHERE usage_geo_id_name NOT LIKE '%федеральный округ%' AND usage_geo_id_name != 'Россия'
GROUP BY usage_geo_id_name
ORDER BY sum_hours DESC
LIMIT 20;

## Задание 2

С активными регионами определились, а какой контент самый популярный? Получите топ-5 книг по суммарному количеству прочитанных и прослушанных часов на мобильных платформах. Также вычислите среднее время чтения и прослушивания в зависимости от типа книги: текст или аудио. Результат должен выглядеть так: название книги, её автор, суммарное время чтения и прослушивания, среднее время чтения текстовой книги, среднее время прослушивания аудиокниги.
В список включайте только те книги, которые используются в обоих форматах. Числовые значения округляйте до двух знаков после точки.

In [ ]:
WITH books AS (
SELECT main_content_id,
       main_content_type,
       main_content_name,
       main_author_name
FROM source_db.content
WHERE (main_content_name, main_author_name) IN (
    SELECT main_content_name, main_author_name
    FROM source_db.content
    GROUP BY main_content_name, main_author_name
    HAVING COUNT(DISTINCT main_content_type) = 2
)
ORDER BY main_content_name, main_author_name
),
audition_books AS (
SELECT b.main_content_id AS main_content_id,
       b.main_content_type AS main_content_type,
       b.main_content_name AS main_content_name,
       b.main_author_name AS main_author_name,
       a.usage_platform_ru AS usage_platform_ru,
       a.hours AS hours
FROM books AS b
JOIN source_db.audition AS a ON b.main_content_id = a.main_content_id 
WHERE a.usage_platform_ru IN ('Букмейт iOS', 'Букмейт Android'))
SELECT main_content_name,
       main_author_name,
       round(SUM(hours), 2) AS sum_hours,
       round(sumIf(hours, main_content_type = 'Book'), 2) AS sum_hours_book,
       round(sumIf(hours, main_content_type = 'Audiobook'), 2) AS sum_hours_audiobook
FROM audition_books
GROUP BY main_content_name, main_author_name
ORDER BY sum_hours DESC
LIMIT 5;

## Задание 3


Составьте топ-10 авторов по суммарной длительности чтения их книг на всех платформах, включая веб. Для каждого автора добавьте количество уникальных текстовых книг (тип контента 'Book' ) и выведите среднюю длительность прослушивания их аудиокниг только на мобильных устройствах. Исключите авторов, у которых нет аудиокниг.

In [ ]:
WITH authors AS (
SELECT main_content_id,
       main_author_name,
       main_content_type
FROM source_db.content 
WHERE main_author_name NOT IN (
    SELECT main_author_name
    FROM source_db.content
    GROUP BY main_author_name
    HAVING COUNT(CASE WHEN main_content_type = 'Audiobook' THEN 1 END) = 0)
),
audition AS (
SELECT a.main_content_id AS main_content_id,
       a.main_author_name AS main_author_name,
       a.main_content_type AS main_content_type,
       au.usage_platform_ru AS usage_platform_ru,
       au.hours AS hours
FROM authors AS a
JOIN source_db.audition AS au ON a.main_content_id = au.main_content_id
WHERE au.usage_platform_ru IN ('Букмейт iOS', 'Букмейт Android', 'Букмейт Web')
)
SELECT DISTINCT main_author_name,
       round(SUM(hours), 2) AS sum_hours,
       countIf(DISTINCT main_content_id, main_content_type = 'Book') AS count_books,
       round(avgIf(hours, usage_platform_ru = 'Букмейт iOS' OR usage_platform_ru = 'Букмейт Android'), 2) AS avg_hours_mobile
FROM audition
GROUP BY main_author_name
ORDER BY sum_hours DESC
LIMIT 10;

## Задание 4

У продакт-менеджера есть предположение, что среди Android-пользователей аудиокниги почти так же популярны, как тексты. А среди iOS-пользователей читателей книг вдвое больше, чем слушателей, если считать по суммарной длительности сессии.
Проверьте предположение менеджера. Для начала выделите три сегмента пользователей:
«Слушатель» — тот, кто преимущественно пользуется аудиокнигами. Прослушивание книг составляет 70% и выше от суммарной длительности сессий.
«Читатель» — преимущественно пользуется текстовыми книгами. Чтение книг — от 70%.
«Оба» — остальные пользователи сервиса.
Исключите пользователей, у которых нет сессий ни с книгами, ни с аудиокнигами, и посчитайте количество пользователей в каждом из сегментов.
На основе полученных данных проверьте предположение менеджера о том, что среди пользователей Android примерно одинаковое количество читателей и слушателей, а на устройствах iOS читателей книг вдвое больше, чем слушателей. Чтобы определить основную платформу пользователя, учитывайте время её использования. Например, если пользователь посещал сервис с двух устройств: два часа на iOS и пять часов на Android, то основной платформой такого пользователя будет Android.

In [ ]:
WITH user_content_stats AS (
    SELECT 
        a.puid,
        a.usage_platform_ru,
        SUM(CASE WHEN c.main_content_type = 'Audiobook' THEN a.hours ELSE 0 END) as audio_hours,
        SUM(CASE WHEN c.main_content_type = 'Book' THEN a.hours ELSE 0 END) as book_hours,
        SUM(a.hours) as total_hours
    FROM source_db.audition a
    JOIN source_db.content c ON a.main_content_id = c.main_content_id
    WHERE c.main_content_type IN ('Audiobook', 'Book') AND a.usage_platform_ru IN ('Букмейт iOS', 'Букмейт Android')
    GROUP BY a.puid, a.usage_platform_ru
),
user_segments AS (
    SELECT 
        puid,
        usage_platform_ru,
        audio_hours,
        book_hours,
        total_hours,
        CASE 
            WHEN audio_hours / total_hours >= 0.7 THEN 'Слушатель'
            WHEN book_hours / total_hours >= 0.7 THEN 'Читатель'
            ELSE 'Оба'
        END as user_segment
    FROM user_content_stats
    WHERE total_hours > 0  
),
user_main_platform AS (
    SELECT 
        puid,
        usage_platform_ru as main_platform
    FROM (
        SELECT 
            puid,
            usage_platform_ru,
            SUM(hours) as platform_hours,
            ROW_NUMBER() OVER (PARTITION BY puid ORDER BY SUM(hours) DESC) as rn
        FROM source_db.audition
        WHERE usage_platform_ru = 'Букмейт Android' OR usage_platform_ru = 'Букмейт iOS'
        GROUP BY puid, usage_platform_ru
    ) ranked
    WHERE rn = 1
),
final_users AS (
    SELECT 
        us.puid,
        ump.main_platform,
        us.user_segment,
        us.audio_hours,
        us.book_hours,
        us.total_hours
    FROM user_segments us
    JOIN user_main_platform ump ON us.puid = ump.puid
    WHERE us.usage_platform_ru = ump.main_platform  
)
SELECT 
    main_platform,
    user_segment,
    COUNT(DISTINCT puid) as user_count
FROM final_users
WHERE user_segment = 'Слушатель' OR user_segment = 'Читатель'
GROUP BY main_platform, user_segment
ORDER BY main_platform, user_segment;

## Задание 5

Изучите, существует ли связь между форматом использования приложения (прослушивание или чтение) и днём недели. Падает ли использование аудиокниг в выходные на всех платформах, включая веб? Чтобы это узнать, для каждого типа контента посчитайте среднее время его использования в рабочие и выходные дни и округлите до целого числа. Используя оконные функции, вы также можете пользоваться и комбинаторами.

In [ ]:
WITH day_of_week AS (
SELECT a.main_content_id AS main_content_id,
       a.msk_business_dt_str AS msk_business_dt_str,
       CASE 
        WHEN toDayOfWeek(toDate(msk_business_dt_str)) IN (6, 7) THEN 'Выходной'
        ELSE 'Рабочий'
    END as day_type,
       a.usage_platform_ru AS usage_platform_ru,
       c.main_content_type AS main_content_type,
       a.hours AS hours
FROM source_db.audition AS a
JOIN source_db.content AS c ON c.main_content_id = a.main_content_id 
WHERE (a.usage_platform_ru = 'Букмейт Android' OR a.usage_platform_ru = 'Букмейт iOS' OR a.usage_platform_ru = 'Букмейт Web') AND (c.main_content_type = 'Book' OR c.main_content_type = 'Audiobook')
)
SELECT main_content_type,
       round(avgIf(hours, day_type = 'Рабочий'), 2) AS avg_hours_weekday,
       round(avgIf(hours, day_type = 'Выходной'), 2) AS avg_hours_weekend
FROM day_of_week
GROUP BY main_content_type;

## Задание 6

Продакт-менеджер хочет отслеживать обновления приложений на Android и iOS. У него есть предположение, что больший процент пользователей iOS используют последнюю версию приложения и в целом чаще его обновляют.
Для начала изучите, у какой части пользователей на текущий момент стоят последние версии приложения на каждой из платформ. Для этого посчитайте последнюю активную версию каждого пользователя и сравните её с последней версией у каждой платформы. Для каждой платформы выведите процент пользователей с последней версией приложения и округлите его до двух знаков после точки.

In [ ]:
WITH platform_latest_version AS (
    SELECT 
        usage_platform_ru,
        argMax(app_version, msk_business_dt_str) as latest_version
    FROM source_db.audition
    WHERE usage_platform_ru IN ('Букмейт iOS', 'Букмейт Android')
        AND app_version IS NOT NULL
        AND trim(app_version) != ''
        AND trim(app_version) != ' '
        AND msk_business_dt_str IS NOT NULL
    GROUP BY usage_platform_ru
),
user_latest_version AS (
    SELECT 
        puid,
        usage_platform_ru,
        argMax(app_version, msk_business_dt_str) as user_last_version
    FROM source_db.audition
    WHERE usage_platform_ru IN ('Букмейт iOS', 'Букмейт Android')
        AND app_version IS NOT NULL
        AND trim(app_version) != ''
        AND trim(app_version) != ' '
        AND msk_business_dt_str IS NOT NULL
    GROUP BY puid, usage_platform_ru
),
user_version_comparison AS (
    SELECT 
        ulv.puid,
        ulv.usage_platform_ru,
        ulv.user_last_version,
        plv.latest_version as platform_latest_version,
        CASE 
            WHEN ulv.user_last_version = plv.latest_version THEN 1
            ELSE 0
        END as is_latest_version
    FROM user_latest_version ulv
    JOIN platform_latest_version plv ON ulv.usage_platform_ru = plv.usage_platform_ru
),
platform_stats AS (
    SELECT 
        usage_platform_ru,
        COUNT(DISTINCT puid) as total_users,
        SUM(is_latest_version) as users_with_latest_version,
        ROUND(SUM(is_latest_version) * 100.0 / COUNT(DISTINCT puid), 2) as percentage_with_latest_version
    FROM user_version_comparison
    GROUP BY usage_platform_ru
)
SELECT 
    usage_platform_ru as platform,
    total_users,
    users_with_latest_version,
    percentage_with_latest_version as percentage_with_latest_version
FROM platform_stats
ORDER BY usage_platform_ru;

## Задание 7

Теперь продакт-менеджер хочет понять, как часто пользователи обновляют приложение на каждой из платформ. Фактом обновления считайте изменение версии у каждого пользователя. Представьте, что любое изменение возможно только в сторону более новой версии.
Проверьте предположение о том, что пользователи iOS чаще обновляют приложение. Посчитайте метрику update_rate, которая покажет среднюю частоту обновлений на пользователя. Округлите её до двух знаков после точки.

In [ ]:
WITH user_updates AS (
    SELECT 
        puid,
        usage_platform_ru,
        COUNT(DISTINCT app_version) - 1 as update_count  -- Вычитаем первоначальную версию
    FROM source_db.audition
    WHERE usage_platform_ru IN ('Букмейт iOS', 'Букмейт Android')
        AND app_version IS NOT NULL
        AND trim(app_version) != ''
        AND trim(app_version) != ' '
    GROUP BY puid, usage_platform_ru
    HAVING COUNT(DISTINCT app_version) > 0  
)
SELECT 
    usage_platform_ru as platform,
    COUNT(DISTINCT puid) as total_users,
    ROUND(SUM(update_count) * 1.0 / COUNT(DISTINCT puid), 2) as update_rate
FROM user_updates
GROUP BY usage_platform_ru
ORDER BY update_rate DESC;

## Задание 8

Новая задача — у коллег есть опасения, что не все книги на тему магии верно размечены с точки зрения категорий.  Считается, что у книги должно быть не больше 3–4 категорий с темами. Необходимо найти все книги на магическую тему, которые при этом не входят в художественную литературу, и проверить, правильно ли они размечены.
Начните с подсчёта книг с тегом «Магия». Выведите количество таких книг в каталоге.

In [ ]:
SELECT 
    'Всего книг с тегом "Магия"' as metric,
    COUNT(DISTINCT main_content_id) as value
FROM source_db.content 
WHERE has(published_topic_title_list, 'Магия')

UNION ALL

SELECT 
    'Книг с магией, но без художественной литературы' as metric,
    COUNT(DISTINCT main_content_id) as value
FROM source_db.content 
WHERE has(published_topic_title_list, 'Магия')
  AND NOT has(published_topic_title_list, 'Художественная литература')

UNION ALL

SELECT 
    'Книг с более чем 4 категориями' as metric,
    COUNT(DISTINCT main_content_id) as value
FROM source_db.content 
WHERE has(published_topic_title_list, 'Магия')
  AND length(published_topic_title_list) > 4

UNION ALL

SELECT 
    'Книг с неправильной разметкой' as metric,
    COUNT(DISTINCT main_content_id) as value
FROM source_db.content 
WHERE has(published_topic_title_list, 'Магия')
  AND (
      length(published_topic_title_list) > 4
      OR NOT has(published_topic_title_list, 'Художественная литература')
  );

## Задание 9

Найдите все книги со словом «магия» в названии, для которых не проставлен тег «Магия». При этом не учитывайте книги с тегом «Художественная литература». Выведите количество таких книг в каталоге.

In [ ]:
SELECT 
    COUNT(DISTINCT main_content_id) as books_count
FROM source_db.content 
WHERE 
    main_content_name ILIKE '%магия%'
    AND NOT has(published_topic_title_list, 'Магия')
    AND NOT has(published_topic_title_list, 'Художественная литература');

## Задание 10

Посчитайте среднее количество категорий у книг с тегом «Магия» и среднее количество категорий у книг в каталоге в целом. Округлите значения до двух знаков после точки. Напомним, что коллегам важно, чтобы у каждой книги было не больше 3–4 категорий. Получится ли не превысить рекомендованного количества?

In [ ]:
SELECT 
    'Книги с тегом "Магия"' as category,
    COUNT(DISTINCT main_content_id) as books_count,
    ROUND(AVG(length(published_topic_title_list)), 2) as avg_categories_count,
    ROUND(MIN(length(published_topic_title_list)), 2) as min_categories,
    ROUND(MAX(length(published_topic_title_list)), 2) as max_categories
FROM source_db.content 
WHERE has(published_topic_title_list, 'Магия')

UNION ALL

SELECT 
    'Все книги в каталоге' as category,
    COUNT(DISTINCT main_content_id) as books_count,
    ROUND(AVG(length(published_topic_title_list)), 2) as avg_categories_count,
    ROUND(MIN(length(published_topic_title_list)), 2) as min_categories,
    ROUND(MAX(length(published_topic_title_list)), 2) as max_categories
FROM source_db.content;

## Задание 11

Продакт-менеджер выяснил, что в приложении одной из мобильных платформ могла возникнуть проблема — длина пользовательской сессии (поле hours_sessions_long ) записывается некорректно, и это происходит как минимум в одной из стран. Чтобы найти аномалию в данных, используйте такую меру дисперсии как коэффициент вариации. Напомним его формулу: коэффициент определяется как отношение стандартного отклонения к среднему. Чем выше этот показатель, тем более подозрительно с точки зрения анализа распределены данные.
Исследуйте коэффициент по странам и мобильным платформам. В какой стране и на какой платформе видна аномалия в данных? Ограничьте выборку одной страной, в которой коэффициент вариации для одной из платформ будет наибольшим.

In [ ]:
SELECT 
    usage_country_name as problem_country,
    usage_platform_ru as problem_platform,
    ROUND(stddevPop(hours_sessions_long) / AVG(hours_sessions_long), 2) as max_variation_coef
FROM source_db.audition
WHERE usage_platform_ru IN ('Букмейт iOS', 'Букмейт Android')
    AND hours_sessions_long > 0
    AND usage_country_name IS NOT NULL
    AND usage_country_name != ''
GROUP BY usage_country_name, usage_platform_ru
HAVING COUNT(*) >= 10
ORDER BY max_variation_coef DESC
LIMIT 1;